# [Deep Learning](https://github.com/M-Sc-AUT/M.Sc-Computer-Architecture/tree/main/Deep%20Learning) - Spring 2024
#### [Reza Adinepour](https://github.com/rezaAdinepour)
#### Stu.ID: 402131055

## Part C:
In this part we should create vector fitures as a polynomial matrix such as: 
$$ x_1x_2, x_1x^2_2, x^3_1x_2, ... $$

and then train again network with this new fitures

for create polynomial fitures i use $\texttt{PolynomialFeatures}$ class from $\texttt{sklearn}$ library

Generate a new feature matrix consisting of all polynomial combinations
of the feaxtures with degree less than or equal to the specified degree.
For example, if an input sample is two dimensional and of the form
$[a, b]$, the degree-2 polynomial features are $[1, a, b, a^2, ab, b^2]$.

so for example, if we have $3\times2$ matrix like bellow:

$$
       X=\begin{bmatrix} 
              0 & 1 \\ 
              2 & 3 \\ 
              4 & 5 
       \end{bmatrix}
$$

after use this class for generate 2 dimention polynomial, in the output we have: 

$$
       X_{new}=\begin{bmatrix} 
              1 & 0 & 1 & 0 & 0 & 1\\
              1 & 2 & 3 & 4 & 6 & 9\\ 
              1 & 4 & 5 & 16 & 20 & 25\\ 
       \end{bmatrix}
$$

### import libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import f1_score


%matplotlib inline

### check GPU availability

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


### Read dataset and data visualization

In [4]:
# read dataset from .txt file and remove first 8 row and set new header with this name: x, y, label
df = pd.read_csv("data.txt", skiprows=8, header=None, names=['x', 'y', 'label'])

# split dataset into the inputs x and the outputs y (labels)
X = df[['x', 'y']]
y = df['label']

print("shape of data frame is:", df.shape)
df

shape of data frame is: (4500, 3)


,x,y,label
0,4.780310,2.698320,0
1,-2.273710,5.750180,0
2,-4.817170,-2.209610,0
3,-9.526940,0.659360,0
4,1.864400,-5.019510,0
...,...,...,...
4495,-4.327500,-0.210256,1
4496,-1.077300,1.279540,1
4497,-2.377350,-3.078130,1
4498,0.508152,0.432983,1


### Convert data to PyTorch tensor and make polynomial fitures